In [1]:
import tensorflow as tf

## Session

In [2]:
matrix1 = tf.constant([[3,3]])
matrix2 = tf.constant([[2], [2]])

In [3]:
product = tf.matmul(matrix1, matrix2)

In [4]:
with tf.Session() as sess:
    result = sess.run(product)
    mat1 = sess.run(matrix1)
    mat2 = sess.run(matrix2)
    print(result, mat1, mat2)

[[12]] [[3 3]] [[2]
 [2]]


## Variable

In [5]:
state = tf.Variable(0, name='counter')

In [6]:
one = tf.constant(1)

In [7]:
new_value = tf.add(state, one)

In [8]:
update = tf.assign(state, new_value)

In [9]:
init = tf.global_variables_initializer()

In [10]:
with tf.Session() as sess:
    sess.run(init)
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

1
2
3


## Placeholder

In [11]:
x1 = tf.placeholder(tf.float32,[1])
x2 = tf.placeholder(tf.float32,[1])

In [12]:
y = tf.multiply(x1, x2)

In [13]:
with tf.Session() as sess:
    print(sess.run(y, feed_dict={x1:[2.], x2:[3.]}))

[ 6.]


## Addlayer

In [43]:
def add_layer(inputs, in_size, out_size, activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size, out_size], dtype=tf.float32))
    biases = tf.Variable(tf.zeros([1, out_size], dtype=tf.float32)) + 0.1
    
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    
    return outputs, Weights, biases

## NN

In [44]:
import numpy as np
from matplotlib import pyplot as plt, rc, rcParams, animation
rc('animation',html='html5')
rcParams['figure.figsize'] = 3, 3

In [45]:
x = np.linspace(-1,1,300,dtype=np.float32)[:, np.newaxis]

In [46]:
y = np.square(x) - 0.5

In [47]:
noise = np.random.normal(0, 0.05, x.shape) # 噪声点，会随机使y上移或下移

In [48]:
y = y + noise

In [49]:
xs = tf.placeholder(tf.float32,[None, 1])
ys = tf.placeholder(tf.float32,[None, 1])

In [50]:
layer1, w1, b1 = add_layer(xs, 1, 20, tf.nn.relu)

In [51]:
prediction,_ ,_ = add_layer(layer1, 20, 1)

In [52]:
loss = tf.reduce_mean(tf.square(ys - prediction))

In [53]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

In [54]:
fig = plt.figure(dpi=100, figsize=(5, 4))
plt.scatter(x,y)
line, = plt.plot(x,x**2,'r-',lw=5)
plt.close()

In [55]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [2]:
def animate(i):
    for i in range(20):
        sess.run(train_step, feed_dict={xs:x, ys:y})
    y_pred = sess.run(prediction, feed_dict={xs:x})
    line.set_ydata(y_pred)
    return line
ani = animation.FuncAnimation(fig, animate, np.arange(0, 80), interval=100)
ani

NameError: name 'animation' is not defined

In [7]:
import tensorflow as tf

def add_layer(inputs, in_size, out_size, activation_function=None):
    with tf.name_scope('Weights'):
        Weights = tf.Variable(tf.random_normal([in_size, out_size], dtype=tf.float32))
    with tf.name_scope('biases'):    
        biases = tf.Variable(tf.zeros([1, out_size]), dtype=tf.float32) + 0.1
    
    with tf.name_scope('Wx_plus_b'):
        Wx_plus_b = tf.matmul(inputs, Weights) + biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    
    return outputs
    
with tf.name_scope('inputs'):
    xs = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='x_inputs')
    ys = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='y_inputs')
    
with tf.name_scope('layer1'):
    layer1, = add_layer(xs, 1, 20, tf.nn.relu)
    
with tf.name_scope('prediction'):
    prediction, = add_layer(layer1, 20, 1)
    
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.square(ys - prediction))
    
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

init = tf.global_variables_initializer()
sess = tf.Session()
writer = tf.summary.FileWriter('/',sess.graph())
sess.run(init)


TypeError: 'Tensor' object is not iterable.